In [ ]:
import math,simplekml,os
import pandas as pd
import geopandas as gpd
import numpy as np
import mytools

# 1-导入数据

In [ ]:
#输出位置
out_file = 'G:/1-规划/评估/覆盖空洞/20200712第四版本/4-相关图层输出/2小区覆盖/'

In [ ]:
data = pd.read_csv('G:/1-规划/评估/覆盖空洞/20200712第四版本/2-程序数据出-过程/1-修正半径后使用的数据/1_修正半径_2周边建筑修正.csv',encoding='gbk')

In [ ]:
data.head(1)

# 参数设置

In [ ]:
data_use = data[['小区CGI(*)', '小区中文名', 'RRU经度', 'RRU纬度', '天线挂高', '方位角',
        '归属区县','所属地区', '区域类型', 'distance', 'h_use', 'distance_use_3']]
#- 标注 -添加标注列
use_columns = ['CGI', 'name', 'lon', 'lat', '基站高度', '方位角',
        '归属区县','所属地区', '区域类型', '标准覆盖半径', '建筑物高度', '修正后覆盖半径']
data_use.columns = use_columns

In [ ]:
#- 变量 - 设置经纬度和参与循环的字段
lon,lat,azimuth,radius,city,region ,use_id= ['lon', 'lat',  '方位角', '修正后覆盖半径','所属地区','归属区县','name']

In [ ]:
#- 颜色 - 只要一个颜色
data_use['colour']='white'

In [ ]:
# #- 颜色 - 按照字符设置颜色
# colour_data = '空洞级别'
# colour_dic = {'大':'red','中':'yellow','小':'blue','微':'white','other':'white'}

In [ ]:
#- 视野 - 在1400*900的分辨率下  view = 70米对应：125公里（差不多能看到武汉主城区）  700对应1200公里（差不多能看到湖北省） 40：65公里
view = 700

# 2-数据整理-将所需的数据添加到列

### 功能模块化

In [ ]:
#添加标注列
de_col = use_columns
data_use['description']=''
for inde_1, name_1 in enumerate(use_columns):
    data_use['linshi']=de_col[inde_1]+' : '+data_use[de_col[inde_1]].astype('str')+'\n'
    data_use['description'] = data_use['description']+data_use['linshi']
    data_use.drop(columns='linshi')

In [ ]:
# 添加扇形的列
data_sectors = mytools.gisn.add_sectors_df(data_use,
    coords=[lon,lat,'d_height',azimuth,radius],
    has_z=False,
    sec_col='geometry',
    shape_dict={'beam': 120, 'per_degree': 10})

In [ ]:
#添加lod的polygon
data_sectors['lod_dis']= view
data_sectors['lon1'] = data_sectors[lon] + data_sectors['lod_dis']*np.sin(45* np.pi/180)*180/( np.pi * 6371229 * np.cos(data_sectors[lat] * np.pi/180))
data_sectors['lat1'] = data_sectors[lat] + data_sectors['lod_dis']*np.cos(45* np.pi/180) / ( np.pi * 6371229 / 180)
data_sectors['lon2'] = data_sectors[lon] + data_sectors['lod_dis']*np.sin(225* np.pi/180)*180/( np.pi * 6371229 * np.cos(data_sectors[lat] * np.pi/180))
data_sectors['lat2'] = data_sectors[lat] + data_sectors['lod_dis']*np.cos(225* np.pi/180) / ( np.pi * 6371229 / 180)
data_sectors['list_data']=[([lon_1, lat_1,lon_2, lat_2]) 
 for lon_1, lat_1,lon_2, lat_2 in zip(data_sectors['lon1'],data_sectors['lat1'],data_sectors['lon2'],data_sectors['lat2'])]

In [ ]:
# #添加颜色列
# df = data_sectors.copy()
# df['colour']=1
# df['colour'] = pd.cut(df['colour'],bins=[-0.1, 0.7, 0.936, 1.1],labels =['red', 'yellow', 'green'])
# df['colour'] = df['colour'].astype('object')
# # df.loc[df[rsrp_count]<100,'colour']='white'

In [ ]:
#最终使用的列
df_use = data_sectors.reindex(columns=[city,region,use_id, 'list_data','colour','description','geometry'])   

In [ ]:
#导出数据
df_use.to_pickle('{}图层制作google使用.data'.format(out_file))

# 生成图层

In [ ]:
def make_kml(data_t,name='红色栅格',cc='ff0000ff',xiankuan=0,namea='grid_id',list_data = 'list_data',description='description',minlodpixels=1):
    style = simplekml.Style()
    #style.linestyle.color = simplekml.Color.changealphaint(150, cc)  # 最终线条上色
    style.polystyle.outline = xiankuan
    style.polystyle.color = simplekml.Color.changealphaint(125, cc )  # 最终形状上色
    lod1 = simplekml.Lod(minlodpixels=1, maxlodpixels=-1,minfadeextent=None, maxfadeextent=None)
    grid_red = kml.newfolder(name=name)
    for grid,list_data ,description_str,geo in zip(data_t[namea],data_t[list_data],data_t[description],data_t['geometry']):
#         pol_r = grid_red.newpolygon(name=grid,outerboundaryis=list_data[0])#栅格使用
        pol_r = grid_red.newpolygon(name=grid,outerboundaryis=list(geo.exterior.coords))
        pol_r.description = description_str
        pol_r.altitudemode = simplekml.AltitudeMode.clamptoground
        lon_dd,lat_dd,lon1_dd,lat1_dd = list_data
        latlonaltbox = simplekml.LatLonAltBox(east =lon_dd ,north=lat_dd ,south=lat1_dd ,west=lon1_dd,
                                              minaltitude=None, maxaltitude=None, altitudemode=None)
        pol_r.region.latlonaltbox = latlonaltbox
        pol_r.region.lod = lod1
        pol_r.style=style

In [ ]:
f = mytools.othern.file_name_paths(out_file)
for name_f in f:
#     name_region = name_f.split('.')[0].split('\\')[1]
    grid_t = pd.read_pickle(name_f)
    grid_groupby = grid_t.groupby([city])
    for name_region,data_region in grid_groupby:
        kml = simplekml.Kml()
#         if not os.path.exists('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/修正覆盖半径/各个地市覆盖图层/{}'.format(name_region)):
#             os.makedirs('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/修正覆盖半径/各个地市覆盖图层/{}'.format(name_region))
        file = '{}宏站覆盖图层_{}.kmz'.format(out_file,name_region)
        for name_c,data_t in data_region.groupby('colour'):
            if 'red' ==name_c:
                make_kml(data_t,name='红色{}个'.format(data_t.shape[0]),cc='ff0000ff',namea=use_id)
            elif 'green' == name_c:
                make_kml(data_t,name='绿色{}个'.format(data_t.shape[0]),cc='ffe6d8ad',namea=use_id)#正常是ff008000
            elif 'yellow' == name_c:
                make_kml(data_t,name='黄色{}个'.format(data_t.shape[0]),cc='ff00ffff',namea=use_id)
            elif 'white' == name_c:
                make_kml(data_t,name='白色{}个'.format(data_t.shape[0]),cc='ffffffff',namea=use_id)
        kml.save(file) if 'kml' in file else kml.savekmz(file, False)
        print('生成图层成功存放在{}'.format(file))

In [ ]:
mytools.sound()